# Measurement optimization data process

This notebook organizes the Jacobian and variance data by choosing the corresponding Jacobian and Var of the candidate measurements, and store them in files that

In [1]:
import numpy as np 
import pickle 

In [2]:
# specify Nz and Ntheta to be Nz*Ntheta
# 10 z locations, 10 theta locations
grid_option = "10*10"

In [3]:
def load_pickle(name): 
    """Used for loading data from pickle file 
    """
    file = open(name, "rb")
    data = pickle.load(file)
    file.close()
    return data
def save_pickle(data, name):
    """Used for saving data to pickle file 
    """
    file = open(name, "wb")
    pickle.dump(data, file)
    file.close()

def read_one_measure(key, path, scale_option =None):
    """Read one measurement Jacobian and Var value
    
    Arguments
    ---------
    key: string, measurement name 
    path: file path 
    scale_option: choose from "scale_parameter", "not_scale_parameter", "scale_para_and_measure"
    """
    # read jaco file
    jaco_file = path +key+"_"+scale_option
    jacobian = load_pickle(jaco_file)
    # read var file
    var_file = path + "Var_"+key+"_"+scale_option
    var = load_pickle(var_file)
    
    return jacobian, var

In [4]:
# grid 20*20 and 10*10 are stored in different path
if grid_option == "20*20":
    read_path = "./jaco_and_var/"
    # length of measurement
    measure_len = [40,800,800,800,800,800,800]
    num_z_grid, num_t_grid=20,20
    # total locations
    total_loc = 800
# grid 10*10
elif grid_option == "10*10":
    read_path = "./jaco_and_var_10t10/"
    # length of each measurement
    measure_len = [20, 200, 200, 180, 200]
    num_z_grid, num_t_grid=10,10
    # total locations
    total_loc = 200

In [5]:
# from here, we reorganize the measurements order 
# this order cannot be changed because we then organize all prices based on this
measure_keys = ["vg", 
                "Tg", 
                "Ts", 
                "P",
                "y_CO2", 
               ]

# number of total measurements
num_rows = sum(measure_len)
# the first index of each measurement
measure_head_row = {}
# each first index is stacked as: 
# vg: index 0 - 19 
# Tg: index 20 - 220 
# ... 
count = 0 
for i, item in enumerate(measure_keys):
    measure_head_row[item] = count 
    count += measure_len[i]
print(measure_head_row)
# total number of parameters
n_para = 5 

{'vg': 0, 'Tg': 20, 'Ts': 220, 'P': 420, 'y_CO2': 600}


In [6]:
def reorganize_jac_and_var(measure_items, scale_option="scale_parameter"):
    """This function extracts Jacobian and variance from stored files, 
    reorganize the data file for chosen candidate measurements 
    
    Arguments
    ---------
    measure_items: measurement names 
    scale_option: choose from "scale_parameter", "not_scale_parameter", "scale_para_and_measure"
    """
    # store the Jacobian matrix and Variance matrix
    all_jac = np.zeros((num_rows, n_para))
    all_var = np.zeros((num_rows, 1))
    
    # loop over measurement items
    for i, item in enumerate(measure_items):
        # locate the first row of the matrix
        head_row = measure_head_row[item]
        # read the jacobian and variance from files
        jaco, var = read_one_measure(item, read_path, scale_option = scale_option)
        
        # if measurement is velocity, only get outlet end locations
        if item == "vg": 
            # we only records z=1 for adsorption, and z=0 for desorption 
            # loop over ads. section outlet
            for j in range(num_t_grid):
                # locate head row
                row_num = head_row + j
                jaco_head = total_loc//2-num_t_grid
                # give value to each parameter
                for p in range(n_para):    
                    all_jac[row_num, p] = jaco[jaco_head+j, p]
                    all_var[row_num] = var[jaco_head+j]
            # loop over des. section outlet
            for j in range(num_t_grid,2*num_t_grid):
                # locate head row
                row_num = head_row + j
                jaco_head = total_loc//2 - num_t_grid
                # give value to each parameter
                for p in range(n_para):    
                    all_jac[row_num, p] = jaco[jaco_head+j, p]
                    all_var[row_num] = var[jaco_head+j]
        # if measurement is pressure
        elif item=="P":
            # for each location
            for j in range(len(jaco)//2-num_t_grid):
                # locate the head row
                row_num = head_row+j
                jaco_head = 0 
                print(row_num, jaco_head+j)
                # loop over each parameter
                for p in range(n_para):
                    all_jac[row_num, p] = jaco[jaco_head+j, p]
                    all_var[row_num] = var[jaco_head+j]
            # for each location in desorption section
            for j in range(len(jaco)//2+num_t_grid, len(jaco)):
                row_num = head_row + j - 2*num_t_grid 
                print(row_num, j)
                # loop over parameter
                for p in range(n_para):
                    all_jac[row_num,p] = jaco[j, p]
                    all_var[row_num] = var[j]
        # for all the other measurements
        else:
            # ads. and des. locations are looped over
            for j in range(len(jaco)):
                # locate head row 
                row_num = head_row + j 
                # loop over parameter
                for p in range(n_para):
                    all_jac[row_num,p] = jaco[j,p]
                    all_var[row_num] = var[j]
                
    return all_jac, all_var
            

In [7]:
all_jacobian, all_variance = reorganize_jac_and_var(measure_keys)
print(measure_head_row)

420 0
421 1
422 2
423 3
424 4
425 5
426 6
427 7
428 8
429 9
430 10
431 11
432 12
433 13
434 14
435 15
436 16
437 17
438 18
439 19
440 20
441 21
442 22
443 23
444 24
445 25
446 26
447 27
448 28
449 29
450 30
451 31
452 32
453 33
454 34
455 35
456 36
457 37
458 38
459 39
460 40
461 41
462 42
463 43
464 44
465 45
466 46
467 47
468 48
469 49
470 50
471 51
472 52
473 53
474 54
475 55
476 56
477 57
478 58
479 59
480 60
481 61
482 62
483 63
484 64
485 65
486 66
487 67
488 68
489 69
490 70
491 71
492 72
493 73
494 74
495 75
496 76
497 77
498 78
499 79
500 80
501 81
502 82
503 83
504 84
505 85
506 86
507 87
508 88
509 89
510 110
511 111
512 112
513 113
514 114
515 115
516 116
517 117
518 118
519 119
520 120
521 121
522 122
523 123
524 124
525 125
526 126
527 127
528 128
529 129
530 130
531 131
532 132
533 133
534 134
535 135
536 136
537 137
538 138
539 139
540 140
541 141
542 142
543 143
544 144
545 145
546 146
547 147
548 148
549 149
550 150
551 151
552 152
553 153
554 154
555 155
556 156
557 

In [8]:
save_pickle(all_jacobian, read_path + "jacobian_"+str(num_z_grid)+"_" +str(num_t_grid))
save_pickle(all_variance, read_path + "variance_"+str(num_z_grid)+"_" +str(num_t_grid))